# Setup Dependencies and Imports
Import required libraries including Semantic Kernel, dataclasses, async libraries, and custom banking APIs.

In [ ]:
# Import required libraries
from dataclasses import dataclass
from datetime import datetime
import asyncio
import os
from typing import List, Optional

# Import Semantic Kernel components
from semantic_kernel import Kernel
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.agents.strategies import KernelFunctionSelectionStrategy
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatHistoryTruncationReducer, ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.connectors.ai.prompt_execution_settings import PromptExecutionSettings

# Import custom banking APIs
from finma_plugin import FinmaPlugin
from seco_plugin import SecoPlugin
from zefix_plugin import ZefixPlugin
from shab_plugin import ShabPlugin
from bank_plugin import BankPlugin
from shared_state import init_sample_data

# Create Core Banking Components
Initialize the Semantic Kernel with Azure Chat Completion service and set up banking plugins (FINMA, SECO, Zefix, SHAB, Bank).

> [!NOTE]  
> The APIs utilized in these labs are intended solely for demonstration purposes and should not be used for any purpose other than experimentation.

In [ ]:
# Create the Semantic Kernel and initialize it with Azure Chat Completion service
def create_kernel() -> Kernel:
    kernel = Kernel()
    
    # Add chat completion service
    service = AzureChatCompletion()
    kernel.add_service(service)
    
    # Add all plugins
    kernel.add_plugin(FinmaPlugin(), plugin_name="finma")
    kernel.add_plugin(SecoPlugin(), plugin_name="seco")
    kernel.add_plugin(ZefixPlugin(), plugin_name="zefix")
    kernel.add_plugin(ShabPlugin(), plugin_name="shab")
    kernel.add_plugin(BankPlugin(), plugin_name="bank")
    
    return kernel

# Initialize the kernel
kernel = create_kernel()

# Initialize sample data
init_sample_data()

# Define Agent System
Create specialized agents (KYC Officer, Account Manager, Risk Officer) with their respective responsibilities and access to specific banking functions.

In [ ]:
# Define agent names
KYC_OFFICER = "KYC_Officer"
ACCOUNT_MANAGER = "Account_Manager"
RISK_OFFICER = "Risk_Officer"

# Define KernelArguments for agents
agent_args = KernelArguments(
    settings=PromptExecutionSettings(
        function_choice_behavior=FunctionChoiceBehavior.Auto(),
    )
)

# Create specialized banking agents
agent_kyc = ChatCompletionAgent(
    service_id=KYC_OFFICER,
    kernel=kernel,
    name=KYC_OFFICER,
    arguments=agent_args,
    instructions=f"""
You are a KYC Officer with access to the following functions:

- finma.search_intermediaries: Search FINMA registry for insurance intermediaries
- seco.check_sanctions: Check if a person/entity is on sanctions list
- zefix.search_companies: Search Swiss company registry
- shab.search_publications: Search official gazette publications

Base all decisions and actions strictly on gathered evidence and API responses, not on pre-existing knowledge.

Process:
1. For new customers:
   - Companies: Use zefix.search_companies and shab.search_publications
   - Insurance intermediaries: Use finma.search_intermediaries
   - All customers: Use seco.check_sanctions
2. Analyze results and determine risk level
3. Make KYC recommendations

Current date: {datetime.now().isoformat()}
"""
)

agent_account = ChatCompletionAgent(
    service_id=ACCOUNT_MANAGER,
    kernel=kernel,
    name=ACCOUNT_MANAGER,
    arguments=agent_args,
    instructions="""
You are an Account Manager with access to the following functions:

- bank.create_account: Create new bank accounts
    - If Company: ONLY if Company was found in registry and is not liquidated!
    - If Person: ONLY if person is not on sanctions list
- bank.get_account: Get account details and status
- bank.freeze_account: Freeze an account with a reason
- bank.unfreeze_account: Unfreeze a previously frozen account

Process:
1. For new accounts:
   - Verify KYC status is approved
   - Use correct account type (individual/company)
   - Include company UID for company accounts
   - Do not create accounts for companies if they were not found in a registry

2. For existing accounts / account reviews:
   - Check account status and details
   - Monitor for suspicious activity
   - Freeze account if suspicious activity detected
   - Unfreeze account when issues are resolved

Always verify customer identity and risk level before operations.
"""
)

agent_risk = ChatCompletionAgent(
    service_id=RISK_OFFICER,
    kernel=kernel,
    name=RISK_OFFICER,
    arguments=agent_args,
    instructions="""
You are a Risk Officer who evaluates overall customer risk and makes final decisions.

Base all decisions and actions strictly on gathered evidence and API responses, not on pre-existing knowledge.

Your responsibilities:
1. Review KYC findings
2. Assess overall risk level
3. Make final decisions on account actions
4. Monitor high-risk situations

You must document all risk-related decisions.
"""
)

# Configure Agent Selection Strategy
Implement the agent selection logic using KernelFunctionFromPrompt to determine which agent should respond based on the conversation context.

In [ ]:
# Configure Agent Selection Strategy
selection_function = KernelFunctionFromPrompt(
    function_name="agent_selection",
    prompt=f"""
{{{{$history}}}}

Based on the chat history above, select the next appropriate agent.
Return ONLY ONE of these names (no other text):
{KYC_OFFICER}
{ACCOUNT_MANAGER}
{RISK_OFFICER}

The KYC officer has access to: 
- finma.search_intermediaries
- seco.check_sanctions
- zefix.search_companies
- shab.search_publications

The Account Manager has access to:
- bank.create_account
- bank.get_account
- bank.freeze_account
- bank.unfreeze_account

The Risk Officer has access to:
- finma.search_intermediaries
- seco.check_sanctions
- zefix.search_companies
- shab.search_publications

Criteria:
- {KYC_OFFICER} handles new customers and verification
- {RISK_OFFICER} handles sanctions and high-risk cases
- {ACCOUNT_MANAGER} handles account operations
"""
)

history_reducer = ChatHistoryTruncationReducer(target_count=10)

# Create the group chat with updated strategy configuration
chat = AgentGroupChat(
    agents=[agent_kyc, agent_account, agent_risk],
    selection_strategy=KernelFunctionSelectionStrategy(
        initial_agent=agent_kyc,
        function=selection_function,
        kernel=kernel,
        history_variable_name="history",  # Match the prompt variable
        agent_variable_name="agents",     # For agent list access
        result_parser=lambda x: str(x).strip() if x else KYC_OFFICER  # Ensure string output
    )
)

# Launch Interactive Interface
Create and launch the chat interface for interacting with the multi-agent system.

In [ ]:
# Create and launch the UI for the multi-agent chat system
from chat_ui import create_chat_interface

# Create the chat interface
ui = create_chat_interface(chat)

# Launch the UI on the specified server and port
ui.launch(server_name="0.0.0.0", server_port=7860)